## Обучение на CBoW модел

Този ноутбук е част от [AI for Beginners Curriculum](http://aka.ms/ai-beginners)

В този пример ще разгледаме обучението на езиков модел CBoW, за да създадем собствено векторно представяне на думи (Word2Vec embedding space). Ще използваме набора от данни AG News като източник на текст.


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Първо нека заредим нашия набор от данни и да дефинираме токенизатор и речник. Ще зададем `vocab_size` на 5000, за да ограничим изчисленията малко.


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## Модел CBoW

CBoW се учи да предсказва дума въз основа на $2N$ съседни думи. Например, когато $N=1$, ще получим следните двойки от изречението *I like to train networks*: (like,I), (I, like), (to, like), (like,to), (train,to), (to, train), (networks, train), (train,networks). Тук първата дума е съседната дума, използвана като вход, а втората дума е тази, която предсказваме.

За да изградим мрежа, която предсказва следващата дума, ще трябва да подадем съседната дума като вход и да получим номера на думата като изход. Архитектурата на мрежата CBoW е следната:

* Входната дума преминава през слой за вграждане (embedding layer). Този слой за вграждане ще бъде нашето Word2Vec вграждане, затова ще го дефинираме отделно като променлива `embedder`. В този пример ще използваме размер на вграждането = 30, въпреки че може да искате да експериментирате с по-големи размерности (реалният Word2Vec има 300).
* Векторът на вграждането след това ще бъде подаден към линеен слой, който ще предсказва изходната дума. Следователно той има `vocab_size` неврони.

За изхода, ако използваме `CrossEntropyLoss` като функция за загуба, ще трябва да предоставим само номерата на думите като очаквани резултати, без едно-горещо кодиране (one-hot encoding).


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## Подготовка на тренировъчни данни

Сега нека програмираме основната функция, която ще изчислява CBoW двойки думи от текст. Тази функция ще ни позволи да зададем размер на прозореца и ще върне набор от двойки - входна и изходна дума. Имайте предвид, че тази функция може да се използва както за думи, така и за вектори/тензори - което ще ни позволи да кодираме текста, преди да го подадем на функцията `to_cbow`.


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


Нека подготвим тренировъчния набор от данни. Ще преминем през всички новини, ще извикаме `to_cbow`, за да получим списъка с двойки думи, и ще добавим тези двойки към `X` и `Y`. За да спестим време, ще разгледаме само първите 10k новини - лесно можете да премахнете ограничението, ако имате повече време за изчакване и искате да получите по-добри вграждания :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

Ще преобразуваме тези данни в един набор от данни и ще създадем dataloader:


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

Ще преобразуваме тези данни в един набор от данни и ще създадем dataloader:


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

Сега нека преминем към действителното обучение. Ще използваме оптимизатора `SGD` с доста висока скорост на обучение. Можете също да опитате да експериментирате с други оптимизатори, като например `Adam`. Ще обучаваме за 10 епохи за начало - и можете да изпълните тази клетка отново, ако искате още по-ниска загуба.


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## Изпробване на Word2Vec

За да използваме Word2Vec, нека извлечем вектори, съответстващи на всички думи в нашия речник:


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

Нека видим, например, как думата **Париж** се кодира в вектор:


In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


Интересно е да се използва Word2Vec за търсене на синоними. Следната функция ще върне `n` най-близки думи до даден вход. За да ги намерим, изчисляваме нормата на $|w_i - v|$, където $v$ е векторът, съответстващ на нашата входна дума, а $w_i$ е кодирането на $i$-тата дума в речника. След това сортираме масива и връщаме съответните индекси, използвайки `argsort`, и вземаме първите `n` елемента от списъка, които кодират позициите на най-близките думи в речника.


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## Основни моменти

Използвайки умни техники като CBoW, можем да обучим модел Word2Vec. Можете също така да опитате да обучите модел skip-gram, който е създаден да предсказва съседната дума, като се даде централната, и да видите колко добре се представя.



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
